In [ ]:
!pip install opencv-contrib-python

In [ ]:
# Import Libraries

import cv2
import cv2 as cv
from cv2 import aruco
from matplotlib import pyplot as plt
import math
from math import atan2, cos, sin, sqrt, pi
import numpy as np

In [ ]:
# init camera
camera = cv2.VideoCapture(0)

In [ ]:
# Grab a single frame of video
ret, frame = camera.read()

In [ ]:
# New sample image:
#img = cv2.imread('/Users/noah/Desktop/python/MobileRobotics/imageproc/project2.png')

# Image from camera:
img = frame

# Display image in RGB 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb);

In [ ]:
# Read / Save image

 
# read image as grey scale
#grey_img = cv2.imread('/home/img/python.png', cv2.IMREAD_GRAYSCALE)
 
# save image
status = cv2.imwrite('/Users/noah/Desktop/python/MobileRobotics/imageproc/project2.png',frame)
 
print("Image written to file-system : ",status)

In [ ]:
# close camera
camera.release()

In [ ]:
import cv2
import numpy as np
from ipywidgets import *
import matplotlib.pyplot as plt

img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_hsv_blur = cv2.medianBlur(img_hsv, 15)


def f (hMin ,hMax ,Smin,Smax ,Vmin,Vmax):
    #img1 = img
    #img = cv2.cvtColor(img1 ,cv2.COLOR_BGR2RGB)  #in matplot image show in rgb format in cv2 it show in bgr format remeber this else you will face problem
    #hsv = cv2.cvtColor(img_rgb,cv2.COLOR_RGB2HSV)# if you are going to use cv2 to show image then cv2.COLOR_BGR2HSV 
    hsv = img_hsv_blur
    Lhsv = np.array([hMin,Smin,Vmin])
    Uhsv = np.array([hMax ,Smax,Vmax])
    mask = cv2.inRange(hsv,Lhsv ,Uhsv)
    r = cv2.bitwise_and(img_rgb,img_rgb,mask =mask)
    
    
    plt.imshow(r)
  
    
interactive_plot = interactive(f,hMin=(0,179),hMax = (0,179),Smin = (0,250),Smax =(0,255),Vmin=(0,255),Vmax=(0,255))

In [ ]:
interactive_plot

In [ ]:
def get_obstacle_position(img, thymio_radius, min_green = np.array([30, 87, 0]), max_green = np.array([90, 255, 255])):
    '''
    Function that outputs vertices of the obstacles (dilated by the Thymio's radius) for the visibility graph path planning

    Inputs: - img : image from camera in BGR format
            - thymio_radius : radius of Thymio in pixels (can be found with the get_thymio_position function)
            - min_green : minimum values for Hue, Saturation and Value for hsv mask
            - max_green : minimum values for Hue, Saturation and Value for hsv mask

    Output: - vertices : array (1 x nb_obstacles x 2) containing list of each obstacle's vertices
            - nb_obstacles : number of obstacles detected (after the dilatation)
            - x_max : length of the image img
            - y_max : width of the image img
    '''    

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_hsv_blur = cv2.medianBlur(img_hsv, 15)
    mask = cv2.inRange(img_hsv_blur, min_green, max_green)
    mask = cv2.erode(mask, None, iterations=2) # 3x3 kernel used
    mask = cv2.dilate(mask, None, iterations=2)
    
    y_max, x_max = mask.shape

    # Expand obstacle size by the radius of the thymio
    dilatation_size = int(thymio_radius*1.2) # add security margin of 20% of thymio's radius
    kernel = cv2.getStructuringElement(cv.MORPH_RECT, (2 * dilatation_size + 1,2 * dilatation_size + 1))
    mask_dilated = cv2.dilate(mask, kernel, iterations = 1)

    contours = cv2.findContours(mask_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2] # RETR_EXTERNAL to get external contour, CHAIN_APPROX_SIMPLE to get geometrical shape 

    output = cv2.bitwise_and(img, img, mask=mask)
    nb_obstacles = 0
    vertices = []
    if len(contours) != 0:
        contours = sorted(contours, key=lambda x:cv2.contourArea(x), reverse=True)
        for c in contours:
            area = cv.contourArea(c)
            #print(area)
            if area > 10000:
                nb_obstacles += 1
                epsilon = 0.015*cv.arcLength(c,True)
                approx = cv.approxPolyDP(c,epsilon,True)
                cv2.drawContours(output, [approx], -1, (0, 255, 255), 20)
                vertices.append(approx)
            else: 
                break

    plt.imshow(output)
    
    return (vertices, nb_obstacles, x_max, y_max)








def get_goal_position(img, min_blue = np.array([70, 80, 0]), max_blue = np.array([120, 255, 255])):
    '''
    Function that outputs goal position from initial camera frame

    Inputs: - img : image from camera in BGR format
            - min_blue : minimum values for Hue, Saturation and Value for hsv mask
            - max_blue : minimum values for Hue, Saturation and Value for hsv mask

    Output: - center : tuple (x,y) representing the center of the goal (in pixels)
            - goal_detected : boolean set to True if goal was detected, False if not
    '''
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_hsv_blur = cv2.medianBlur(img_hsv, 15)
    mask = cv2.inRange(img_hsv_blur, min_blue, max_blue)
    mask = cv2.erode(mask, None, iterations=2) # 3x3 kernel used
    mask = cv2.dilate(mask, None, iterations=2)
    
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2] # RETR_EXTERNAL to get external contour, CHAIN_APPROX_SIMPLE to get geometrical shape 
    output = cv2.bitwise_and(img, img, mask=mask)
    
    x_center = 0
    y_center = 0
    goal_detected = False
    if len(contours) != 0:
        
        # find the biggest countour (c) by the area
        c = max(contours, key = cv2.contourArea)
        M = cv2.moments(c)
        x_center = int(M['m10']/M['m00'])
        y_center = int(M['m01']/M['m00'])
        if cv2.contourArea(c) > 5000:
            goal_detected = True
            (x,y),radius = cv2.minEnclosingCircle(c)
            center = (int(x),int(y))
            radius = int(radius)
            cv2.circle(output,center,radius,(0,255,0),10) 
            
    plt.imshow(output)
          
        
    return center, goal_detected






def dist(pt1, pt2):
    distance = ( (pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2 )**0.5
    return distance



def get_thymio_position(img, img_output):
    
    '''
    Function that outputs Thymio position, radius and orientation from camera frames using Aruco marker detection
    Parts of code taken from https://mecaruco2.readthedocs.io/en/latest/notebooks_rst/Aruco/aruco_basics.html

    Inputs: - img : image from camera in BGR format
            - img_output : copy of image img on which will be drawn Thymio's contours

    Output: - center : tuple (x,y) representing the center of the Thymio (in pixels)
            - radius : radius of the Thymio (in pixels)
            - angle : orientation of the Thymio (in radians)
            - thymio_detected : boolean set to True if Thymio was detected, False if not
    '''
    
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    parameters =  aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    frame_markers = aruco.drawDetectedMarkers(img_output, corners, ids)
    
    
    thymio_detected = False
    center = (0,0)
    radius = 0
    angle = 0
    
    if np.all(ids != None):
        for i in range(len(ids)):
            if ids[i] == 0:
                thymio_detected = True
                c = corners[i][0]

                top_center = (c[0:2, 0].mean(),c[0:2, 1].mean())
                bottom_center = (c[2:4, 0].mean(), c[2:4, 1].mean())
                left_center = (c[0:4:3, 0].mean(), c[0:4:3, 1].mean())
                right_center = (c[1:3, 0].mean(), c[1:3, 1].mean())
                points = np.array([ [c[2,0],c[2,1]], [c[3,0],c[3,1]], [left_center[0],left_center[1]], [right_center[0],right_center[1]] ])

                square_center = (int(c[:, 0].mean()), int(c[:, 1].mean()))
                radius = int(max( dist(square_center,c[0]), dist(square_center,c[1]), dist(square_center,c[2]), dist(square_center,c[3])))
                angle = math.atan2(top_center[1]-bottom_center[1],top_center[0]-bottom_center[0])
                
                
                center = (int(points[:, 0].mean()), int(points[:, 1].mean()))
                cv2.arrowedLine(img_output, (center[0],center[1]), (int(top_center[0]),int(top_center[1])), (255,255,0), 10)
                cv2.circle(img_output,square_center,radius,(255,0,0),10)

                break

        
    return center, radius, angle, thymio_detected








